# Predictive Thyroid Disease Project - Modeling

To make a predictive model for thyroid cancer patients to see how likely after treatment their cancer will reoccur.

'Response to treatment' is considered if the cancer comes back or not

# Initial Set Up

In [1]:
# General Data Manipulation
import pandas as pd
import numpy as np


# Import my custom library 
import MyCustDataSciLib as MyCustDataSciLib

In [2]:
# Load your cleaned CSV file
df = pd.read_csv('4_Thyroid_Disease_FeatureEngineer.csv') # Change it to Thyroid_Disease_EDA

In [3]:
df.head()

,gender_m_dummy_encoded,smoking_yes_dummy_encoded,hx_smoking_yes_dummy_encoded,hx_radiotherapy_yes_dummy_encoded,thyroid_function_2cat_normal_dummy_encoded,focality_uni-focal_dummy_encoded,m_m1_dummy_encoded,stage_2cat_late_dummy_encoded,recurrence_yes_dummy_encoded,kmode_cluster_3cat_one_hot_encoded,...,risk_label_encoded,stage_4cat_label_encoded,response_3cat_label_encoded,stage_label_encoded,t_4cat_label_encoded,pathology_label_encoded,n_label_encoded,response_label_encoded,thyroid_function_3cat_label_encoded,thyroid_function_label_encoded
0,False,False,False,False,True,True,False,False,False,0,...,2,0,1,0,0,2,0,2,1,2
1,False,False,True,False,True,True,False,False,False,0,...,2,0,0,0,0,2,0,1,1,2
2,False,False,False,False,True,True,False,False,False,0,...,2,0,0,0,0,2,0,1,1,2
3,False,False,False,False,True,True,False,False,False,0,...,2,0,0,0,0,2,0,1,1,2
4,False,False,False,False,True,False,False,False,False,0,...,2,0,0,0,0,2,0,1,1,2


In [4]:
# check the columns data type before edit
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 383 entries, 0 to 382
Data columns (total 88 columns):
 #   Column                                                            Non-Null Count  Dtype
---  ------                                                            --------------  -----
 0   gender_m_dummy_encoded                                            383 non-null    bool 
 1   smoking_yes_dummy_encoded                                         383 non-null    bool 
 2   hx_smoking_yes_dummy_encoded                                      383 non-null    bool 
 3   hx_radiotherapy_yes_dummy_encoded                                 383 non-null    bool 
 4   thyroid_function_2cat_normal_dummy_encoded                        383 non-null    bool 
 5   focality_uni-focal_dummy_encoded                                  383 non-null    bool 
 6   m_m1_dummy_encoded                                                383 non-null    bool 
 7   stage_2cat_late_dummy_encoded                        

# Model 1 